In [ ]:
# import needed libraries
import numpy as np
import os
import matplotlib.pyplot as plt
from google.colab import drive
import matplotlib.image as mpimg
import pandas as pd
from IPython import display
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import cv2
from zipfile import ZipFile 
from pathlib import Path
import sys
import tensorflow as tf
from PIL import Image
from PIL import ImageChops
import zipfile
from zipfile import ZipFile
from google.colab.patches import cv2_imshow
import pandas as pd

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Read Dataset**

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Classification/wdbc.data')
dataset.head(10)

,842302,M,17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189
0,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.013080,0.01860,0.013400,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.18600,0.2750,0.08902
1,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.040060,0.03832,0.020580,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.24300,0.3613,0.08758
2,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.074580,0.05661,0.018670,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.25750,0.6638,0.17300
3,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.024610,0.05688,0.018850,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.16250,0.2364,0.07678
4,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.007510,0.033450,0.03672,0.011370,0.02165,0.005082,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.17410,0.3985,0.12440
5,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,0.4467,0.7732,3.180,53.91,0.004314,0.013820,0.02254,0.010390,0.01369,0.002179,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368
6,84458202,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,0.5835,1.3770,3.856,50.96,0.008805,0.030290,0.02488,0.014480,0.01486,0.005412,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510
7,844981,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,0.3063,1.0020,2.406,24.32,0.005731,0.035020,0.03553,0.012260,0.02143,0.003749,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.20600,0.4378,0.10720
8,84501001,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,0.2976,1.5990,2.039,23.94,0.007149,0.072170,0.07743,0.014320,0.01789,0.010080,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.22100,0.4366,0.20750
9,845636,M,16.02,23.24,102.70,797.8,0.08206,0.06669,0.03299,0.03323,0.1528,0.05697,0.3795,1.1870,2.466,40.51,0.004029,0.009269,0.01101,0.007591,0.01460,0.003042,19.19,33.88,123.80,1150.0,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452


**a.Preprocessing**

In [ ]:
X_df = dataset.drop(dataset.columns[[0,1]], axis=1)
Y_df = dataset[dataset.columns[1]]
X = X_df.values
Y = Y_df.values

In [ ]:
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
X_after_normalization = preprocessing.normalize(X)
X_after_selection = SelectKBest(f_classif, k= 20).fit_transform(X_after_normalization, Y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_after_selection, Y , test_size = 57)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(511, 20) (57, 20) (511,) (57,)


**b.Classification**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
def logistic_regression(x_train, y_train):
  clf = LogisticRegression(random_state=0)
  clf.fit(x_train, y_train)
  return clf
clf = logistic_regression(x_train, y_train)
print(clf.get_params)
y_pred = clf.predict(x_test)
print("Prediction: ",y_pred)
print("Labels: ", y_test)
acc= accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
precision = precision_score(y_test, y_pred, labels=['M','B'], average='micro')
print("Precision:",precision)
recall = recall_score(y_test, y_pred, labels=['M','B'], average='micro')
print('Recall:',recall)
score = f1_score(y_test, y_pred, labels=['M','B'], average='micro')
print('F-Measure:', score)

<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)>
Prediction:  ['B' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B'
 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B'
 'B' 'B' 'B']
Labels:  ['B' 'B' 'B' 'B' 'M' 'M' 'M' 'M' 'B' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'B'
 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'B'
 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'B'
 'B' 'M' 'M']
Accuracy:  0.7719298245614035
Precision: 0.7719298245614035
Recall: 0.7719298245614035
F-Measure: 0.7719298245614035


In [ ]:
from sklearn.model_selection import GridSearchCV
def hyperParameters_tuning(x_train, y_train):
  logistic = LogisticRegression()
  C = [1000, 10, 1.0, 0.1, 0.01, 100]
  hyperparameters = dict(C=C)
  clf = GridSearchCV(logistic, hyperparameters)
  clf.fit(x_train, y_train)
  return clf
clf_tuning = hyperParameters_tuning(x_train, y_train)
print('Best C:', clf_tuning.best_estimator_.get_params()['C'])
y_pred = clf_tuning.predict(x_test)
print("Prediction: ",y_pred)
print("Labels: ", y_test)
acc= accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
precision = precision_score(y_test, y_pred, labels=['M','B'], average='micro')
print("Precision:",precision)
recall = recall_score(y_test, y_pred, labels=['M','B'], average='micro')
print('Recall:',recall)
score = f1_score(y_test, y_pred, labels=['M','B'], average='micro')
print('F-Measure:', score)

Best C: 100
Prediction:  ['B' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M'
 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B'
 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B'
 'B' 'M' 'M']
Labels:  ['B' 'B' 'B' 'B' 'M' 'M' 'M' 'M' 'B' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'B'
 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'B'
 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'B'
 'B' 'M' 'M']
Accuracy:  0.8596491228070176
Precision: 0.8596491228070176
Recall: 0.8596491228070176
F-Measure: 0.8596491228070176


**Helped_links:**

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
https://chrisalbon.com/machine_learning/model_selection/hyperparameter_tuning_using_grid_search/
https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
https://machinelearningmastery.com/precision-recall-and-f-measure-for-imbalanced-classification/